In [1]:
import os

import fiona
import geopandas as gpd
import gpxpy
import numpy as np
import pandas as pd
from multiprocess.pool import Pool
from tqdm.notebook import tqdm, trange
import warnings
import geopy.distance
from shapely import wkt
import json

warnings.filterwarnings('ignore')

fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'

storage = "/Volumes/easystore/Drones/"

flight_details = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/gpx-with-census-data.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)

flight_blocks = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../data/outputs/outputs_drone-seconds-block-count-per-flight.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
def wkt_to_geojson(wkt_str):
    geom = wkt.loads(wkt_str)
    coordinates = [list(coord) for coord in geom.exterior.coords]
    geojson = {
        "type": "Polygon",
        "coordinates": [coordinates]
    }
    return json.dumps(geojson)

# Apply the conversion to the geometry column
cv_blocks = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../data/outputs/outputs_cv-blocks-geometry.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

In [2]:
cv_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../data/outputs/outputs_cv-flight-manifest.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)

def build_flight_heatmap(idx):
    posada = flight_details[flight_details['id']==idx]

    posada['timestamp'] = pd.to_datetime(posada['sequence'])
    posada['timestamp'] = posada['timestamp'] + pd.Timedelta(hours=9)
    
    posada['timestamp'] = posada['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    
    posada['GEOID20'] = "0" + posada['geoid'].astype(str)
    posada_blocks = cv_blocks[cv_blocks['GEOID20'].isin(posada['GEOID20'])]
    posada_blocks['geometry_json'] = posada_blocks['geometry'].apply(wkt_to_geojson)
    
    posada_blocks.to_csv(f'./{idx}-flight.csv',index=False)
    posada.to_csv(f'./{idx}-blocks.csv',index=False) 

Loading data: 0it [00:00, ?it/s]

In [20]:
build_flight_heatmap('ba36a26e27ee10435fb8f4cb65c77d35')

In [21]:
cv_flights[cv_flights['type'].astype(str).str.contains('nan',case=False)].sample(50)

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department,clean-type,match
5192,32.60070869076499,-117.0486754400252,8:53am,15287402409971cb2cc3bbffe7d4e87c,2022-08-23,8:53am,NaN,NaN,NaN,cvpd,NaN,NaN
6605,32.5929569265532,-117.075397395406,12:03am,944ba7005c581e278f6132f430c98c50,2023-03-07,12:03am,"27th Street &, Faivre St",NaN,NaN,cvpd,NaN,NaN
7376,32.6062427794189,-117.033195962459,11:24pm,df57fcdabb5af593b3616f4650f8e4df,2023-04-29,11:24pm,1500 Oleander Ave,NaN,NaN,cvpd,NaN,NaN
1226,32.64017571,-116.99921622,12:37pm,743138f75f4dabd78f4aaca48fae1414,2021-09-02,12:37pm,"Racquetball courts, 900 Otay Lakes Rd",NaN,NaN,cvpd,NaN,NaN
5686,32.61829294,-116.96933826,4:02pm,f89a6b02205a6f9f5d4dad35ac891c22,2022-09-24,4:02pm,1710 Millenia Ave,NaN,NaN,cvpd,NaN,NaN
8380,32.6399762946639,-117.082702485861,1:55pm,fba3329776264f5945f11702fdcb4799,2023-06-29,1:55pm,315 4th Ave,NaN,NaN,cvpd,NaN,NaN
3197,32.61976972,-117.02297217,5:31pm,ac00b337286d7a6b3abf29359ff8875a,2022-03-26,5:31pm,751 Medical Center Ct,NaN,NaN,cvpd,NaN,NaN
1465,32.638800630901,-117.013215042891,12:52pm,bd064fcb228569bffddff518cedac35f,2021-09-17,12:52pm,850 Paseo Ranchero,NaN,NaN,cvpd,NaN,NaN
570,32.64015719,-116.99921129,4:33pm,4daa5e5a1bf0bb212fb633f367ed09a0,2021-07-30,4:33pm,"Racquetball courts, 900 Otay Lakes Rd",NaN,NaN,cvpd,NaN,NaN
735,32.61737362,-117.07164715,1:28pm,302e1ec61207f422ee1e32797d9045c9,2021-08-06,1:28pm,330 Moss St,NaN,NaN,cvpd,NaN,NaN


In [11]:
di['fn'] = di['id'].apply(lambda x: f"/Volumes/easystore/Drones/flights/kml/{x}.kml")


/var/folders/4c/vsvqyby107s_6_0qp0lykjwm0000gn/T/ipykernel_10237/3440793394.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  di['fn'] = di['id'].apply(lambda x: f"/Volumes/easystore/Drones/flights/kml/{x}.kml")


In [12]:
compiled_flights = []
for _, row in di.iterrows():
    try:
        d = gpd.read_file(row['fn'], driver='KML')


        d['id'] = row['id']
        d['type'] = row['type']
        d['incident_id'] = row['incident_id']
        d['address_map'] = row['address_map']
        d['date'] = row['date']
        d['time'] = row['time_s']

        compiled_flights.append(d)
    except:
        print("nah")


    

In [20]:
pd.read_csv('/Volumes/easystore/Drones/flights/gpx/9dff050e6b677f1a514405d52deb303e.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/easystore/Drones/flights/gpx/9dff050e6b677f1a514405d52deb303e.csv'

In [17]:

coordinates_df=[]
for _, row in di.iterrows():

    d = gpd.read_file(row['fn'], driver='KML')

    for point in d['geometry'].coords:
        # Append the name, latitude, and longitude to the coordinates DataFrame
        coordinates_df.append({
            'name': row['Name'],
            'latitude': point[1],
            'longitude': point[0],
            'altitude': point[2],
        })



KeyError: 'geometry'

In [16]:
d

,Name,Description,geometry,id,type,incident_id,address_map,date,time
0,Home,,POINT Z (-121.81571 39.64256 47.63155),9dff050e6b677f1a514405d52deb303e,Outside assist - missing person,2308030048,2700 block Burdick Road,2023-08-03,4:57am
1,Airdata.com,,"LINESTRING Z (-121.81571 39.64256 47.63155, -1...",9dff050e6b677f1a514405d52deb303e,Outside assist - missing person,2308030048,2700 block Burdick Road,2023-08-03,4:57am


In [13]:
compiled = pd.concat(compiled_flights)


In [14]:
compiled.to_csv('./test.csv')

In [18]:
rdf = gpd.GeoDataFrame( pd.concat( dfs, ignore_index=True) )
rdf.shape[0]

2000

In [19]:
rdf.drop_duplicates().to_file("test.geojson", driver='GeoJSON')


In [20]:
rdf.drop_duplicates()

,Name,Description,geometry,id,type,address
0,Home,,POINT Z (-117.08268 32.64000 22.10523),2adccc56aa72db328879f1fafeaafcb3,Person Down,100 Bonita Rd
1,Airdata.com,,"LINESTRING Z (-117.08268 32.64000 22.10523, -1...",2adccc56aa72db328879f1fafeaafcb3,Person Down,100 Bonita Rd
2,Home,,POINT Z (-117.07166 32.61742 34.82301),e90e2d6bae17030e1e1d9c6dcca0b4a2,Unknown Problem,500 Anita St.
3,Airdata.com,,"LINESTRING Z (-117.07166 32.61742 34.82301, -1...",e90e2d6bae17030e1e1d9c6dcca0b4a2,Unknown Problem,500 Anita St.
4,Home,,POINT Z (-117.08270 32.63998 22.13710),eaaec83162027023bf4a950deaa8a919,Weapon Threat,Broadway/ E St.
...,...,...,...,...,...,...
1995,Airdata.com,,"LINESTRING Z (-117.07164 32.61742 34.79365, -1...",4c8d2d1e7a90dc8bc0a36177e31a5c5c,Collision,I-5/ Palomar St.
1996,Home,,POINT Z (-117.08273 32.63996 22.14651),45d5862d6b272aacba5cc5c12ea1b6d3,Auto Theft,745 H
1997,Airdata.com,,"LINESTRING Z (-117.08273 32.63996 22.14651, -1...",45d5862d6b272aacba5cc5c12ea1b6d3,Auto Theft,745 H
1998,Home,,POINT Z (-117.07166 32.61745 34.88751),be2db64e0805627e3cc250db4c92ccba,Welfare Check,1400 Hilltop Dr.
